In [4]:
import numpy as np

In [11]:
root = 'Elevation'

csv_file = root+'.csv'
import pandas as pd
my_csv = pd.read_csv(csv_file, header=None, names=['x', 'y', 'elevation'])
#column = my_csv.column_name
# you can also use my_csv['column_name']

In [12]:
x = my_csv.values[:,0]
y = my_csv.values[:,1]
dem_elevation = my_csv.values[:,2] 

In [13]:
import os

# Get NetCDF
#infile = NetCDFFile(name_in, netcdf_mode_r) 

#if verbose: print('Reading data from %s' % (name_in))

easting_max = np.max(x)
northing_max = np.max(y)
easting_min = np.min(x)
northing_min = np.min(y)
easting_min, northing_min, easting_max, northing_max


xllcorner = easting_min # Easting of lower left corner
yllcorner = northing_min  # Northing of lower left corner
#cellsize = float(infile.cellsize)
NODATA_value = -9999

#dem_elevation = infile.variables['elevation']

zone = -1 
false_easting = 0.0
false_northing = 0.0



# Text strings
projection = 'wgs84'
datum = 'UTM'
units = 'm'

no_points = len(dem_elevation)

from anuga.file.netcdf import NetCDFFile
from anuga.config import netcdf_mode_r, netcdf_mode_w, netcdf_mode_a, netcdf_float

ptsname = root + '.pts'


#print('Store to NetCDF file %s' % ptsname)

# NetCDF file definition
outfile = NetCDFFile(ptsname, netcdf_mode_w)

# Create new file
outfile.institution = 'Geoscience Australia'
outfile.description = 'NetCDF pts format for compact and portable ' \
                      'storage of spatial point data'

# Assign default values
#if easting_min is None: easting_min = xllcorner
#if easting_max is None: easting_max = xllcorner + dem_len
#if northing_min is None: northing_min = yllcorner
#if northing_max is None: northing_max = yllcorner + nrows*cellsize


#print easting_min, easting_max, northing_min, northing_max

# Compute offsets to update georeferencing
easting_offset = xllcorner - easting_min
northing_offset = yllcorner - northing_min

# Georeferencing
outfile.zone = zone
outfile.xllcorner = easting_min # Easting of lower left corner
outfile.yllcorner = northing_min # Northing of lower left corner
outfile.false_easting = false_easting
outfile.false_northing = false_northing

outfile.projection = projection
outfile.datum = datum
outfile.units = units

outfile.createDimension('number_of_points', no_points)
outfile.createDimension('number_of_dimensions', 2) #This is 2d data

# Variable definitions
outfile.createVariable('points', netcdf_float, ('number_of_points',
                                                'number_of_dimensions'))
outfile.createVariable('elevation', netcdf_float, ('number_of_points',))

# Get handles to the variables
points = outfile.variables['points']
elevation = outfile.variables['elevation']

points[:,0] = x - easting_min
points[:,1] = y - northing_min
elevation[:] = dem_elevation

outfile.close()


Store to NetCDF file Elevation.pts
